In [97]:
import pandas as pd
import time
import random
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options
from getpass import getpass
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common import NoSuchElementException, ElementNotInteractableException

## Webscrapping Ali Express with selenium

In [138]:
url = "https://www.aliexpress.com/p/calp-plus/index.html?spm=a2g0o.categorymp.allcategoriespc.21.68b34Eec4Eecl8&categoryTab=beauty_%2526_health"

driver = webdriver.Edge()
driver.get(url)
driver.maximize_window()

body = driver.find_element(By.TAG_NAME, "body")

for i in range(9):
    body.send_keys(Keys.PAGE_DOWN)
    time_2wait = random.uniform(2,2.5)*2
    time.sleep(time_2wait)

html = driver.page_source

soup = BeautifulSoup(html)
products = soup.find_all("div", attrs = {"class":"_2FypS"})

#we need clicklable element

all_products = driver.find_elements(By.CLASS_NAME, "_2FypS")

In [139]:
len(products)

90

create a function to fetch number of product review. Which is store inside each product page. So the goal is to open each product page and get the data store it and then close the page.

In [140]:
def get_prod_click(prod):
    prod.click()
    driver.switch_to.window(driver.window_handles[-1])
    time_2wait = random.uniform(1,2.5)*2
    time.sleep(time_2wait)
    html_pp = driver.page_source
    soup_pp = BeautifulSoup(html_pp)

    try:
        n_reviews = soup_pp.find_all("a", attrs = {"class":"reviewer--reviews--cx7Zs_V"})[0].get_text().split()[0]
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        return n_reviews
        
    except:
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        return 0

In [141]:
product_names = []
product_prices = []
product_sold = []
product_reviews = []

for prod, all_prod in zip(products, all_products):

        prod_name = prod.find_all("h3", attrs = {"class":"nXeOv"})[0].get_text()
        prod_price = prod.find_all("div", attrs = {"class":"U-S0j"})[0].get_text()
        prod_sold = prod.find_all("span", attrs = {"class":"Ktbl2"})[0].get_text()
        prod_review = get_prod_click(all_prod)

        product_names.append(prod_name)
        product_prices.append(prod_price)
        product_sold.append(prod_sold)
        product_reviews.append(prod_review)


        my_dict = {"names": product_names,
                "prices": product_prices,
                "sold":product_sold,
                "n_reviews":product_reviews}
        df = pd.DataFrame.from_dict(my_dict, orient="columns")
        #df.to_csv("ali_express_BEAUTY_&_HEALTH.csv") 

In [142]:
df

,names,prices,sold,n_reviews
0,500 Handmade Premade Lash Fans Bouquet Eyelash...,￡1.17,297 sold,37
1,20D Loose Fans Eyelash Extensions Pointy Slim ...,￡1.17,309 sold,49
2,Original Curl Hair Styling Fixing Spray Dry Fr...,￡4.25,194 sold,17
3,3 In 1 Instant Colouring Shampoo Herbal Brunet...,￡9.29,11 sold,0
4,H.C.F. TRIPLE SERUM 50ml,￡14.09,6 sold,0
...,...,...,...,...
85,24ml Power Grip Primer Gel 4% Niacinamide Brig...,￡3.93,42 sold,2
86,Qianyan 16 Colors Eyeshadow Pearlescence Explo...,￡21.65,181 sold,49
87,60pcs BIOAQUA Collagen Eye Mask Moisturizing F...,￡1.17,700+ sold,94
88,Peptide Dark Circles Removal Eye Cream Eye Bag...,￡1.17,289 sold,52
